In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import math
from random import random
import matplotlib.pyplot as plt
import seaborn as sns
import operator
from functools import reduce
from tqdm import tqdm
import matplotlib.animation as ani
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
import random
random.seed(12345)

In [3]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
plt.rcParams["font.family"] = "Times New Roman"
plt.style.use('seaborn-whitegrid')
plt.style.use('seaborn-poster')
plt.style.use('seaborn-dark-palette')
plt.rcParams["mathtext.fontset"] = "cm"

In [4]:
def accuracy_plot(true, pred):
    plt.scatter(true, pred, s = 10, c = -true, cmap='viridis')  
    plt.ylabel('predicted migration intensity(birds km-3)')
    plt.xlabel('actual migration intensity(birds km-3)')
    x = np.arange(0, int(max(true)))
    plt.fill_between(x, x*0.9, x*1.1, alpha = 0.3, where=None, interpolate=False, step=None, data=None)
    plt.show()

In [5]:
def hyperparameter_tuning(df_forecasting, coef, radar):
    
    #print(df_forecasting)
    X = df_forecasting.iloc[:,1:]
    y = df_forecasting.iloc[:,0]
    x_train, x_valid = X[:int(coef * (len(X)/30))*30], X[int(coef * (len(X)/30))*30:]
    y_train, y_valid = y[:int(coef * (len(y)/30))*30], y[int(coef * (len(y)/30))*30:]
    
    #param_test1 = {'n_estimators':[10, 50, 100, 150, 200]}
    #model = RandomForestRegressor()
    
    param_grid = [{'n_estimators': range(30, 151, 10), 'random_state':[100]}]
                  #{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},]

    forest_reg = RandomForestRegressor(n_jobs = -1)
    grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
 
    grid_search.fit(x_train, y_train)
    print(grid_search.best_params_)
    
    pred = grid_search.best_estimator_.predict(x_valid)
    pred = pd.Series(pred, index = y_valid.index)
    
    return pred, grid_search.best_params_

def drop_altitude(data):
    df = []
    for i in tqdm(np.arange(0, len(data), 30)):
        df.append(data[i:i+30].sum())
    return df


In [17]:
def hyperparameter_tuning1(df_forecasting, coef, radar, n_estimators, max_depth):
    
    #print(df_forecasting)
    X = df_forecasting.iloc[:,1:]
    y = df_forecasting.iloc[:,0]
    x_train, x_valid = X[:int(coef * (len(X)/30))*30], X[int(coef * (len(X)/30))*30:]
    y_train, y_valid = y[:int(coef * (len(y)/30))*30], y[int(coef * (len(y)/30))*30:]
    
    #param_test1 = {'n_estimators':[10, 50, 100, 150, 200]}
    #model = RandomForestRegressor()
    
    param_grid = [{'n_estimators': n_estimators, 'max_depth':max_depth, 'random_state':[100]}]
                  #{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},]

    forest_reg = RandomForestRegressor(n_jobs = -1)
    grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
 
    grid_search.fit(x_train, y_train)
    print(grid_search.best_params_)
    
    pred = grid_search.best_estimator_.predict(x_valid)
    pred = pd.Series(pred, index = y_valid.index)
    
    return pred, grid_search.best_params_

# weather with altitude and mean

## n_estimators

In [31]:
radars = pd.read_csv('../dataset/radar.csv').values.reshape(1, -1)
radars = radars[0]
print(radars)

['KABR' 'KABX' 'KAKQ' 'KAMA' 'KAMX' 'KAPX' 'KARX' 'KATX' 'KBBX' 'KBGM'
 'KBHX' 'KBIS' 'KBLX' 'KBMX' 'KBOX' 'KBRO' 'KBUF' 'KBYX' 'KCAE' 'KCBW'
 'KCBX' 'KCCX' 'KCLE' 'KCLX' 'KCRP' 'KCXX' 'KCYS' 'KDAX' 'KDDC' 'KDFX'
 'KDGX' 'KDIX' 'KDLH' 'KDMX' 'KDOX' 'KDTX' 'KDVN' 'KDYX' 'KEAX' 'KEMX'
 'KENX' 'KEPZ' 'KESX' 'KEVX' 'KEWX' 'KEYX' 'KFCX' 'KFDR' 'KFDX' 'KFFC'
 'KFSD' 'KFSX' 'KFTG' 'KFWS' 'KGGW' 'KGJX' 'KGLD' 'KGRB' 'KGRK' 'KGRR'
 'KGSP' 'KGWX' 'KGYX' 'KHDX' 'KHGX' 'KHNX' 'KHTX' 'KICT' 'KICX' 'KILN'
 'KILX' 'KIND' 'KINX' 'KIWA' 'KIWX' 'KJAX' 'KJGX' 'KJKL' 'KLBB' 'KLCH'
 'KLIX' 'KLNX' 'KLOT' 'KLRX' 'KLSX' 'KLTX' 'KLVX' 'KLWX' 'KLZK' 'KMAF'
 'KMAX' 'KMHX' 'KMKX' 'KMLB' 'KMOB' 'KMPX' 'KMQT' 'KMRX' 'KMSX' 'KMTX'
 'KMUX' 'KMVX' 'KNKX' 'KNQA' 'KOAX' 'KOHX' 'KOKX' 'KOTX' 'KPAH' 'KPBZ'
 'KPDT' 'KPUX' 'KRAX' 'KRGX' 'KRIW' 'KRLX' 'KRTX' 'KSFX' 'KSGF' 'KSHV'
 'KSJT' 'KSOX' 'KSRX' 'KTBW' 'KTFX' 'KTLH' 'KTLX' 'KTWX' 'KTYX' 'KUDX'
 'KUEX' 'KVNX' 'KVTX' 'KVWX' 'KYUX']


In [ ]:
coef = 0.5
order_lags = 3
#result = pd.DataFrame(columns=('radar', 'mse', 'r2', 'explained_variance', 'n_estimators'))
result = pd.read_csv('../dataset/result_mean_estimator1.csv')
for radar in tqdm(radars):
    df_forecasting = pd.read_csv('../dataset/regressor_mean1/regressor_mean%i_%s.csv'%(order_lags, radar))
    pred, par = hyperparameter_tuning(df_forecasting, coef, radar)
    pred = drop_altitude(pred)
    true = drop_altitude(df_forecasting['linear_eta'].values)
    test_y = true[(len(true)-len(pred)):]
    train_y = true[:(len(true)-len(pred))]
    #rfm.line_plot1(ts_1 = test_y, ts_label_1 = rfm.radar, ts_2 = pred_y, ts_label_2 = '%s - Forecasting'%rfm.radar, title = '%s forecasting (testing data)'%rfm.radar, m = 'weather_lags5')
    #rfm.line_plot2(ts_1 = test_y, ts_label_1 = '%s observed'%rfm.radar, ts_2 = pred_y, ts_label_2 = '%s - Forecasting'%rfm.radar, ts_3 = train_y, ts_label_3 = '%s training'%rfm.radar, title = '%s forecasting(training + testing)'%rfm.radar, m = 'weather_lags%i'%j)
    var = explained_variance_score(test_y, pred)
    mse = mean_squared_error(test_y, pred)
    r2 = r2_score(test_y, pred)
    print(order_lags, var, mse, r2)
    result = result.append(pd.DataFrame({'radar':[radar],
                                         'mse':[np.round(mse, 3)],
                                         'r2':[np.round(r2, 3)],
                                         'explained_variance':[np.round(var, 3)],
                                         'n_estimators':[par['n_estimators']]}),
                           ignore_index = True)
    result.to_csv('../dataset/result_mean_estimator1.csv', index = False)
    pd.DataFrame(pred).to_csv('../dataset/pred_mean1/pred_mean_%i_opt_%s.csv'%(order_lags, radar), index = False)

100%|██████████| 455/455 [00:00<00:00, 7775.62it/s]

  1%|          | 1/135 [04:43<10:32:44, 283.32s/it]

{'n_estimators': 30, 'random_state': 100}
3 0.42650765773949595 441756767.03292763 0.42347545931953545



100%|██████████| 455/455 [00:00<00:00, 9665.62it/s]

  1%|▏         | 2/135 [09:51<11:01:05, 298.23s/it]

{'n_estimators': 50, 'random_state': 100}
3 0.5486403669561486 126356754.68635319 0.5485417103542828



100%|██████████| 455/455 [00:00<00:00, 7243.86it/s]

100%|██████████| 910/910 [00:00<00:00, 120131.46it/s]

...

 0.79273208135504 533677438.18989 0.7886006503378536
{'n_estimators': 60, 'random_state': 100}
3 0.4316064447383289 796667007.7486355 0.4220989830471702



100%|██████████| 455/455 [00:00<00:00, 8705.85it/s]

 19%|█▉        | 26/135 [1:52:13<7:39:34, 252.97s/it]

{'n_estimators': 30, 'random_state': 100}
3 0.5864421337637824 809931740.9270011 0.5856290892340146



100%|██████████| 455/455 [00:00<00:00, 8109.91it/s]

 20%|██        | 27/135 [1:56:47<7:46:43, 259.30s/it]

{'n_estimators': 100, 'random_state': 100}
3 0.6765193714677382 194364730.24691176 0.6662824129681655


## max_depth

In [ ]:
order_lags = 3
#result = pd.DataFrame(columns=('radar', 'mse', 'r2', 'explained_variance', 'n_estimators'))
n_estimators = pd.read_csv('../dataset/result_mean_estimator.csv')
result = pd.DataFrame(columns=('radar', 'mse', 'r2', 'explained_variance', 'n_estimators', 'max_depth'))
for radar in tqdm(radars):
    df_forecasting = pd.read_csv('../dataset/regressor_mean/regressor_mean%i_%s.csv'%(order_lags, radar))
    n_esti = n_estimators[n_estimators['radar']==radar][['n_estimators']].values[0]
    #print(n_esti)
    pred, par = hyperparameter_tuning1(df_forecasting, coef, radar, n_esti, range(8, 16))
    pred = drop_altitude(pred)
    true = drop_altitude(df_forecasting['linear_eta'].values)
    test_y = true[(len(true)-len(pred)):]
    train_y = true[:(len(true)-len(pred))]
    #rfm.line_plot1(ts_1 = test_y, ts_label_1 = rfm.radar, ts_2 = pred_y, ts_label_2 = '%s - Forecasting'%rfm.radar, title = '%s forecasting (testing data)'%rfm.radar, m = 'weather_lags5')
    #rfm.line_plot2(ts_1 = test_y, ts_label_1 = '%s observed'%rfm.radar, ts_2 = pred_y, ts_label_2 = '%s - Forecasting'%rfm.radar, ts_3 = train_y, ts_label_3 = '%s training'%rfm.radar, title = '%s forecasting(training + testing)'%rfm.radar, m = 'weather_lags%i'%j)
    var = explained_variance_score(test_y, pred)
    mse = mean_squared_error(test_y, pred)
    r2 = r2_score(test_y, pred)
    print(order_lags, var, mse, r2)
    result = result.append(pd.DataFrame({'radar':[radar],
                                         'mse':[np.round(mse, 3)],
                                         'r2':[np.round(r2, 3)],
                                         'explained_variance':[np.round(var, 3)],
                                         'n_estimators':n_esti, 
                                         'max_depth':[par['max_depth']]}),
                           ignore_index = True)
    result.to_csv('../dataset/result_mean_depth.csv', index = False)
    pd.DataFrame(pred).to_csv('../dataset/pred_mean/pred_mean_%i_opt_%s.csv'%(order_lags, radar), index = False)

# Configuration

In [22]:
# Generate the configuration file with best performance parameters
import json
def WriteJson(JsonFile, data, radars):
    with open(JsonFile,'w',encoding = 'GBK') as f1:
        str1 = {'lags':str(3), 'random_state':str(100)}
        f1.writelines(json.dumps(str1, ensure_ascii=False)+'\n')
        for i in range(len(data)):
            str1 ={'radar':data.iloc[i,0], 'n_estimators':str(data.iloc[i,4])}
                   #'max_depth':str(data.loc(radar, 'max_depth'))}
            f1.writelines(json.dumps(str1, ensure_ascii=False)+'\n')


In [23]:
radars = pd.read_csv('../dataset/radar.csv').values.reshape(1, -1)
radars = radars[0]
result = pd.read_csv('../dataset/result_mean_estimator.csv')
WriteJson('config.json', result, radars)